In [1]:
import numpy as np
import json
import itertools
import math
import sys

In [2]:
input = 'size0'
file = open(input,'r')
Ag_lst = []
lines = file.readlines()
for line in lines:
    line = line.rstrip()
    particle_data = [float(x) for x in line.split()]
    coord = particle_data[0:]
    Ag_lst.append(coord)

Ag_tmp1  = np.asarray(Ag_lst)
Ag_tmp2  = Ag_tmp1[Ag_tmp1[:,0].argsort()]
Ag_array = Ag_tmp2[:,2:]

In [3]:
dlta = 0.0
num_particle = Ag_array.shape[0]
max_x = max(Ag_array[:, 0])
max_y = max(Ag_array[:, 1])
max_z = max(Ag_array[:, 2])
max_coord = max(max_x, max_y, max_z)
max_radius = max(Ag_array[:, 3])
grid_size = 2.01 * max_radius
grid_number = int(math.ceil(max_coord / grid_size))
max_index = pow(grid_number, 3) - 1
SYS_thk = max_z + max_radius

In [4]:
print(num_particle)
print(max_x)
print(max_y)
print(max_z)
print(max_coord)
print(max_radius)
print(SYS_thk)

1493
19.601409478334
19.601743789273
9.595290804394
19.601743789273
0.4002
9.995490804394


In [9]:
# Helper function
def grid_index_lst(coord):
    return [int(e / grid_size) for e in coord]

def grid_index(lst):
    """
    Converts a list of index to a 1D index
    :param lst: 3D grid_box index.
    :return: grid_box index
    """
    return lst[0] + grid_number * lst[1] + grid_number * grid_number * lst[2]


def neighbor_grid_index(lst):
    neighbor_list = []
    x_index = [lst[0] - 1, lst[0], lst[0] + 1]
    y_index = [lst[1] - 1, lst[1], lst[1] + 1]
    z_index = [lst[2] - 1, lst[2], lst[2] + 1]
    x_index = [e for e in x_index if 0 <= e < grid_number]
    y_index = [e for e in y_index if 0 <= e < grid_number]
    z_index = [e for e in z_index if 0 <= e < grid_number]
    index_lst = [x_index, y_index, z_index]
    for comb in itertools.product(*index_lst):
        neighbor_list.append(grid_index(comb))
    return neighbor_list


def distance(a, b):
    return np.linalg.norm(a-b)


def type_of_particle(i):
    if i == num_particle:
        return 'Target'
    return 'Ag'

In [10]:
# Assign particles to grid boxes.
index_map = {i: [] for i in range(max_index + 1)}
for i in range(num_particle):
    coord_i = Ag_array[i, :3]
    index_i = grid_index(grid_index_lst(coord_i))
    index_map[index_i].append(i)

count = sum(len(e) for e in index_map.values())
# Build adjacent lists.
adj = {i: {} for i in range(num_particle + 1)}  # dummy node as target.
vertical_dist = dict()
particle_rads = dict()
particle_type = dict()

vertical_dist[num_particle] = 0  # For dummy node.
particle_type[num_particle] = 'Target'
particle_rads[num_particle] = 0.

In [11]:
mxGap = 0.
for i in range(num_particle):
    particle_type[i] = type_of_particle(i)
    radius_i = Ag_array[i, 3]
    coord_i = Ag_array[i, :3]
    z_coord = Ag_array[i, 2]
    vertical_dist[i] = z_coord
    particle_rads[i] = radius_i
    if z_coord <= radius_i+dlta:
        adj[i][num_particle] = [z_coord, 0, 'Target']

    grid_index_i = grid_index_lst(coord_i)
    neighbor_box_i = neighbor_grid_index(grid_index_i)
    for j in neighbor_box_i:
        for k in index_map[j]:
            if k == i:
                continue
            radius_k = Ag_array[k, 3]
            coord_k = Ag_array[k, :3]
            dist = distance(coord_i, coord_k)
            overlap = dist - (radius_i + radius_k)
            if overlap <= dlta:
                adj[i][k] = [dist, overlap, type_of_particle(k)]
                if overlap < mxGap:
                   mxGap = overlap

In [12]:
print(particle_type)

{1493: 'Target', 0: 'Ag', 1: 'Ag', 2: 'Ag', 3: 'Ag', 4: 'Ag', 5: 'Ag', 6: 'Ag', 7: 'Ag', 8: 'Ag', 9: 'Ag', 10: 'Ag', 11: 'Ag', 12: 'Ag', 13: 'Ag', 14: 'Ag', 15: 'Ag', 16: 'Ag', 17: 'Ag', 18: 'Ag', 19: 'Ag', 20: 'Ag', 21: 'Ag', 22: 'Ag', 23: 'Ag', 24: 'Ag', 25: 'Ag', 26: 'Ag', 27: 'Ag', 28: 'Ag', 29: 'Ag', 30: 'Ag', 31: 'Ag', 32: 'Ag', 33: 'Ag', 34: 'Ag', 35: 'Ag', 36: 'Ag', 37: 'Ag', 38: 'Ag', 39: 'Ag', 40: 'Ag', 41: 'Ag', 42: 'Ag', 43: 'Ag', 44: 'Ag', 45: 'Ag', 46: 'Ag', 47: 'Ag', 48: 'Ag', 49: 'Ag', 50: 'Ag', 51: 'Ag', 52: 'Ag', 53: 'Ag', 54: 'Ag', 55: 'Ag', 56: 'Ag', 57: 'Ag', 58: 'Ag', 59: 'Ag', 60: 'Ag', 61: 'Ag', 62: 'Ag', 63: 'Ag', 64: 'Ag', 65: 'Ag', 66: 'Ag', 67: 'Ag', 68: 'Ag', 69: 'Ag', 70: 'Ag', 71: 'Ag', 72: 'Ag', 73: 'Ag', 74: 'Ag', 75: 'Ag', 76: 'Ag', 77: 'Ag', 78: 'Ag', 79: 'Ag', 80: 'Ag', 81: 'Ag', 82: 'Ag', 83: 'Ag', 84: 'Ag', 85: 'Ag', 86: 'Ag', 87: 'Ag', 88: 'Ag', 89: 'Ag', 90: 'Ag', 91: 'Ag', 92: 'Ag', 93: 'Ag', 94: 'Ag', 95: 'Ag', 96: 'Ag', 97: 'Ag', 98: 'Ag', 99: